In [1]:
!pwd

/home/jupyter/drone/transfer


In [3]:
%cd .. 
%cd coco
!bash get_coco_dataset_images.sh # will only use plane images

/home/jupyter/drone
/home/jupyter/drone/coco
--2019-08-23 17:43:19--  https://pjreddie.com/media/files/train2014.zip
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13510435630 (13G) [application/zip]
Saving to: ‘train2014.zip’

train2014.zip       100%[===================>]  12.58G  18.3MB/s    in 8m 48s  

2019-08-23 17:52:07 (24.4 MB/s) - ‘train2014.zip’ saved [13510435630/13510435630]

--2019-08-23 17:52:07--  https://pjreddie.com/media/files/val2014.zip
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6645013297 (6.2G) [application/zip]
Saving to: ‘val2014.zip’

val2014.zip         100%[===================>]   6.19G  29.1MB/s    in 4m 13s  

2019-08-23 17:56:20 (25.1 MB/s) - ‘val2014.zip’ saved [6645013297/664501

In [4]:
!bash unzip_annotations.sh 

Fix archive (-F) - assume mostly intact archive
 copying: instances_train2014.json
  error:  invalid compressed data to inflate instances_train2014.json


In [1]:
import time
import glob
import torch
import os 
from IPython.display import Image, clear_output
print('PyTorch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

PyTorch 1.1.0 _CudaDeviceProperties(name='Tesla P100-PCIE-16GB', major=6, minor=0, total_memory=16280MB, multi_processor_count=56)


In [2]:
!pip install cython
!pip install nvidia-ml-py3
!pip install -U -r requirements.txt

  Stored in directory: /home/jupyter/.cache/pip/wheels/e4/1d/06/640c93f5270d67d0247f30be91f232700d19023f9e66d735c7
Successfully built nvidia-ml-py3
    100% |████████████████████████████████| 20.3MB 2.2MB/s 
Requirement already up-to-date: opencv-python in /opt/anaconda3/lib/python3.7/site-packages (from -r requirements.txt (line 7)) (4.1.0.25)
    100% |████████████████████████████████| 748.9MB 58kB/s 
Retrying (Retry(total=4, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ProtocolError('Connection aborted.', OSError("(104, 'ECONNRESET')"))': /simple/matplotlib/
    100% |████████████████████████████████| 13.1MB 5.1MB/s 
    100% |████████████████████████████████| 1.5MB 24.5MB/s 
    100% |████████████████████████████████| 51kB 20.5MB/s 
    100% |████████████████████████████████| 3.8MB 11.6MB/s 
Requirement already up-to-date: future in /opt/anaconda3/lib/python3.7/site-packages (from -r requirements.txt (line 13)) (0.17.1)
    100% |███████████████

In [1]:
#download weights
%cd weights
!bash download_yolov3_weights.sh
%cd ..

/home/jupyter/drone/transfer/weights
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   388    0   388    0     0   7918      0 --:--:-- --:--:-- --:--:--  7918
100  236M    0  236M    0     0   201M      0 --:--:--  0:00:01 --:--:--  291M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   388    0   388    0     0   7760      0 --:--:-- --:--:-- --:--:--  7918
100  236M    0  236M    0     0   221M      0 --:--:--  0:00:01 --:--:--  302M
Will not apply HSTS. The HSTS database must be a regular and non-world-writable file.
ERROR: could not open HSTS store at '/home/jupyter/.wget-hsts'. HSTS will be disabled.
--2019-08-23 21:39:33--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com 

In [8]:
    #train starting with darknet
#!sudo python3 train.py --cfg cfg/plane-drone.cfg --data data/plane_drone_train_val.data --batch-size 40 --epochs 300 && mv results.txt results_b40_e300.txt
    #or uncomment selection
    #transfer learning
!sudo python3 train.py --transfer --cfg cfg/plane-drone.cfg --data data/plane_drone_train_val.data --batch-size 40 --epochs 300 && mv results.txt results_transfer_b40_e300.txt

Namespace(accumulate=2, batch_size=40, bucket='', cache_images=False, cfg='cfg/plane-drone.cfg', data='data/plane_drone_train_val.data', epochs=300, evolve=False, img_size=416, img_weights=False, multi_scale=False, nosave=False, notest=False, num_workers=16, rect=False, resume=False, transfer=True, xywh=False)
Using CUDA device0 _CudaDeviceProperties(name='Tesla P100-PCIE-16GB', total_memory=16280MB)

data/plane_drone_train.txt
[]
Model Summary: 222 layers, 6.15291e+07 parameters, 37695 gradients

     Epoch   gpu_mem   GIoU/xy        wh       obj       cls     total   targets  img_size
     0/299     1.02G       2.1         0      5.16      49.8        57        42       416: 100%|█| 18/18 [00:12<00:00,  1.35it/s]
data/plane_drone_val.txt
[]
                         Class    Images   Targets         P         R       mAP        F1: 100%|█| 8/8 [00:09<00:00,  1.37s/it]
                           all       301       418         0         0         0         0

     Epoch   gpu_mem   GIo

In [2]:
#resumes from transfer learning with a full train, thus not all layers expect the 3 yolo layers are frozen anymore
#!sudo python3 train.py --resume --cfg cfg/plane-drone.cfg --data data/plane_drone_train_val.data --batch-size 40 --epochs 400 && mv results.txt results_resume_b40_e3_400.txt

Namespace(accumulate=2, batch_size=40, bucket='', cache_images=False, cfg='cfg/plane-drone.cfg', data='data/plane_drone_train_val.data', epochs=400, evolve=False, img_size=416, img_weights=False, multi_scale=False, nosave=False, notest=False, num_workers=16, rect=False, resume=True, transfer=False, xywh=False)
Using CUDA device0 _CudaDeviceProperties(name='Tesla P100-PCIE-16GB', total_memory=16280MB)

data/plane_drone_train.txt
[]
Model Summary: 222 layers, 6.15291e+07 parameters, 6.15291e+07 gradients

     Epoch   gpu_mem   GIoU/xy        wh       obj       cls     total   targets  img_size
   312/399     7.79G     0.358         0     0.157   0.00163     0.517        42       416: 100%|█| 18/18 [00:30<00:00,  2.36s/it]
data/plane_drone_val.txt
[]
                         Class    Images   Targets         P         R       mAP        F1: 100%|█| 8/8 [00:13<00:00,  1.86s/it]
                           all       301       418     0.248     0.851      0.65     0.381

     Epoch   gpu_mem

In [7]:
!sudo python3 test.py --data data/plane_drone_train_val.data --cfg cfg/plane-drone.cfg --weights weights/best.pt --img-size 416 --batch-size 40

Namespace(batch_size=40, cfg='cfg/plane-drone.cfg', conf_thres=0.001, data='data/plane_drone_train_val.data', img_size=416, iou_thres=0.5, nms_thres=0.5, save_json=False, weights='weights/best.pt')
Using CUDA device0 _CudaDeviceProperties(name='Tesla P100-PCIE-16GB', total_memory=16280MB)

data/plane_drone_val.txt
[]
                         Class    Images   Targets         P         R       mAP        F1: 100%|█| 8/8 [00:19<00:00,  2.48s/it]
                           all       301       418    0.0434     0.928     0.667    0.0821
                         drone       301       152    0.0247     0.954     0.646    0.0482
                      airplane       301       266     0.062     0.902     0.688     0.116
Exception ignored in: <function WeakValueDictionary.__init__.<locals>.remove at 0x7f2af66d0f28>
Traceback (most recent call last):
  File "/usr/lib/python3.5/weakref.py", line 117, in remove
TypeError: 'NoneType' object is not callable


In [6]:
!sudo python3 -c "from utils import utils; utils.plot_results()"  #see results.png for all results graph

In [1]:
!python3 detect.py

Namespace(cfg='cfg/plane-drone.cfg', conf_thres=0.3, data='data/plane_drone_train_val.data', fourcc='mp4v', half=False, images='../drone-vs-plane/images/val-test-all', img_size=416, inference_output='inference_output', nms_thres=0.5, webcam=False, weights='weights/best.pt')
Using CUDA device0 _CudaDeviceProperties(name='Tesla P100-PCIE-16GB', total_memory=16280MB)

image 1/301 ../drone-vs-plane/images/val-test-all/351.jpg: 256x416 2 drones, Done. (0.037s)
image 2/301 ../drone-vs-plane/images/val-test-all/352.jpg: 256x416 3 drones, Done. (0.020s)
image 3/301 ../drone-vs-plane/images/val-test-all/353.jpg: 256x416 1 airplanes, Done. (0.018s)
image 4/301 ../drone-vs-plane/images/val-test-all/354.jpg: 256x416 1 drones, Done. (0.018s)
image 5/301 ../drone-vs-plane/images/val-test-all/355.jpg: 192x416 4 drones, Done. (0.022s)
image 6/301 ../drone-vs-plane/images/val-test-all/356.jpg: Done. (0.017s)
image 7/301 ../drone-vs-plane/images/val-test-all/357.jpg: 256x416 5 drones, Done. (0.025s)
ima